In [171]:
import numpy as np

## Part 1

# Batch mode training using least squares - supervised learning of network weights. 


- Implement a radial basis function network from scratch. 

- The network will be used to approximate sin(2x) and square(2x) functions




In [174]:
## Support functions for the evaluation

def getTrainSet(func = 'sin2x', stepSize = 0.1, noise = True):
    ## Returns an 2 x N array of a training set
    # Row 0 = inputs and row 1 = targets¨
    # stepSize is taken as input, range is fixed to 0 --> 2pi. 
    N = int(np.floor(2 * np.pi/stepSize)) #Number of datapoints
    
    train = np.zeros((2,N))
    inputs = np.arange(0, N)
    np.random.shuffle(inputs)
    
    if (noise):
        noise = 1
    else: 
        noise = 0
    
    for step, i in enumerate(inputs):
        train[0, i] = step*stepSize # Input: will be for example 0, 0.1, 0.2 .. 2pi etc.. 
        if (func == 'sin2x'): 
            train[1, i] = np.sin(2*step*stepSize) + noise * np.random.normal(0, np.sqrt(0.1)) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..
        elif (func == 'step2x'): 
            train[1, i] = np.sign(np.sin(2*step*stepSize)) + noise * np.random.normal(0, np.sqrt(0.1)) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..

    return train.T

def getTestSet(func = 'sin2x', stepSize = 0.1):
    ## Returns an 2 x N array of a training set
    # Row 0 = inputs and row 1 = targets¨
    # stepSize is taken as input, range is fixed to 0 --> 2pi. 
    N = int(np.floor(2 * np.pi/stepSize)) #Number of datapoints
    
    test = np.zeros((2,N))

    for step in range(N):
        test[0, step] = step*stepSize+0.05 # Input: will be for example 0.05, 0.15, 0.25 .. 2pi´0.05 etc.. 
        if (func == 'sin2x'): 
            test[1, step] = np.sin(2*step*stepSize+0.05) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..
        elif (func == 'step2x'): 
            test[1, step] = np.sign(np.sin(2*step*stepSize+0.05)) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..

    return test.T

def getBallisticTrainSet():
    data = np.loadtxt('data/ballist.dat', dtype=float)
    inp = []
    target = []
    for d in data[:,0:2]:
        inp.append((d[0], d[1]))
    
    for d in data[:,2:4]:
        target.append((d[0], d[1]))
    inp = np.array(inp)
    target = np.array(target)
    
    return inp, target

def getBallisticTestSet():
    data = np.loadtxt('data/balltest.dat', dtype=float)
    inp = []
    target = []
    for d in data[:,0:2]:
        inp.append((d[0], d[1]))
    
    for d in data[:,2:4]:
        target.append((d[0], d[1]))
    inp = np.array(inp)
    target = np.array(target)
    return inp, target



In [177]:
class RBF:
    def __init__(self, n = 12, variance = 0.1, learning_rate = 0.1, maxinput_x = 1, maxinput_y = 1):
        self.n = n # the number of nodes
        self.variance = 0.1 ## Same variance for all nodes
        # self.units = np.random.rand(1, n) * maxinput # random unit position in the input space
        # self.units = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]) # unit positions 
        
        # Place units with even spaces. each has an x and y coordinate. 
        x_cords = np.arange(0, maxinput_x, maxinput_x/(self.n)) 
        print(x_cords)
        self.units = []
        # the units are an array of tuples
        for i in range(self.n):
            self.units.append((x_cords[i], maxinput_y/2))
        
        
        self.units = np.array(self.units)

        self.n = self.units.shape[0]
        self.w = []
        #for i in range(n): 
        #    self.w.append((np.random.rand(), np.random.rand()))
        #self.w = np.array(self.w)
        self.w = np.random.rand(self.n) # random weights for each node
        self.lr = learning_rate
        print("Initiated weights: ")
        print(self.w)
        #
        #print("Initiated units: ")
        #print(self.units)
        #print(self.n)
    #
    def error(self, f_approx, f):
        # the average error (with direction +/-)
        if not (f_approx.shape == f.shape): 
            raise Exception('f_approx and f shapes mismatch. f_approx: {}, f: {} '.format(f_approx.shape, f.shape))

        # error = (phi(x) - target)^2
        return np.average(f_approx - f)
    
    def res_error(self, f_approx, f):
        # the residual error (absolute)
        if not (f_approx.shape == f.shape): 
            raise Exception('f_approx and f shapes mismatch. f_approx: {}, f: {} '.format(f_approx.shape, f.shape))

        # error = (phi(x) - target)^2
        return np.average(np.abs(f_approx - f))            
    
    def predict(self, inp):
        # takes an input inp and returns predictions
        # do f^ = phi(x) * w.T
        x = inp#.dot(np.ones((2, self.n))) # m x 1 * 1 x n --> m x n
        
        # 1 x n * n x 2
        phi_x = []
        for i in range(self.n): # for each node
            res = self.phi_tup(x, i)
            phi_x.append(res)
        phi_x = np.array(phi_x).T
        f_approx = np.array(phi_x.dot(self.w))
        print("phi_x:")
        print(phi_x.shape)
        print("self.w")
        print(self.w.shape)
        print("Predicted: ")
        print(f_approx)
        print("from input:")
        print(inp)
        return f_approx
    
    def fit_lsq(self, inp, f):
        # x is the input. Shape: inputs x 1
        # f is the true value of the function (aka the target), same shape
        if not (inp.shape == f.shape): 
            raise Exception('inp and f shapes mismatch. inp: {}, f: {} '.format(inp.shape, f.shape))
    

        # We want x to be a matrix with shape: inputs x neurons
        x = inp.dot(np.ones((1, self.n))) # inp x 1 * 1 x n --> inp x n
        
        
        # get phi(x)
        self.phi_x = self.phi_matrix(x)

        # we obtain the w that minimizes the error by solving: 
        # phi(x).T * phi(x) * w = phi(x).T * f
        # --> w = (phi(x).T * phi(x))^-1 * phi(x).T * f        
        self.w = np.linalg.inv(self.phi_x.T.dot(self.phi_x)).dot(self.phi_x.T).dot(f)
        
    
    def fit_delta(self, inp, f):
        # x is the input. Shape: inputs x 1
        # f is the true value of the function (aka the target), same shape
        if not (inp.shape == f.shape): 
            raise Exception('inp and f shapes mismatch. inp: {}, f: {} '.format(inp.shape, f.shape))
            
        self.phi_x = []
        # compute phi(x)
        for i, unit in enumerate(self.units):
            self.phi_x.append((self.phi(inp[0], unit[0]), inp[1], unit[1]))
        
        self.phi_x = np.array(self.phi_x)
        
        # make a prediction
        f_approx = self.predict(inp)
        
        # compute the error
        e = self.error(f_approx, f)
        
        # find delta w
        self.dw = -1 * self.lr * e * self.phi_x.T
        
        print(self.dw.shape)
        #update weights
        self.w += self.dw
            
    def cl_vanilla_update(self, sample, learning_rate):
        # find the "winner" of the units (the one closest to the random sample)
        unit_distance = np.abs(self.units - sample)
        winner_idx = np.argmin(unit_distance)
        
        # print("CL. Random Sample: {} Winner: {}".format(sample, self.units[winner_idx]))
        
        # how far away is the winner's position from the sample? 
        dp = -1 * learning_rate * (self.units[winner_idx] - sample)
        
        # update the winner's position to be slightly closer to the random sample
        self.units[winner_idx] += dp

    def cl_multi_update(self, sample, learning_rate, winners = 5):
        # by looking for multiple winners, we avoid dead units. 
        
        # find multiple "winners" of the units (the one closest to the random sample)
        unit_distance = np.abs(self.units - sample)
        
        for i in range(winners):
            winner_idx = np.argmin(unit_distance)
            unit_distance[winner_idx] = np.inf # set this distance to inf so its not selected again     
            
            # how far away is the winner's position from the sample? 
            dp = -1 * learning_rate * (self.units[winner_idx] - sample)
            
            # Checked w. this print, seems to work
            # print("CL. Random Sample: {} Winner: {}. Diff: {}. Moving {} to: {}".format(sample, self.units[winner_idx], dp, winner_idx, self.units[winner_idx]+dp))

            # update the winner's position to be slightly closer to the random sample
            self.units[winner_idx] += dp
            

    def phi(self, x, i): 
        return np.exp((-(np.linalg.norm(x-i))**2)/(2*np.square(self.variance)))
        
    def phi_tup(self, tup, i):
        unit = self.units[i]
        phi_tup = np.array((self.phi(tup[0], unit[0]), self.phi(tup[1], unit[1])))
        return phi_tup

In [178]:
def delta(n, variance = 0.1, learning_rate = 0.1, epochs = 20, winners = 0):
    # Get datasets
    train_inp, train_target = getBallisticTrainSet()
    print("train inputs example:")
    print(train_inp[0:5])

    # Get test set
    test_inp, test_target = getBallisticTestSet()
    
    model = RBF(n, variance, learning_rate, maxinput_x = np.max(train_inp[0] * 1.5), maxinput_y = np.max(train_inp[1] * 1.5)) # Taking 50% extra input space
    

    for epoch in range(epochs): 

        # Shuffle data
        idx = np.random.permutation(train_inp.shape[0])
        train_inp = train_inp[idx]
        train_target = train_target[idx]
        
        for i, inp in enumerate(train_inp):
            sample = inp
            target = train_target[i]
            model.fit_delta(sample, target)
        
        if (winners > 0): 
            # Update units through competetive learning
            rand = np.random.choice(train_inp[:, 0])
            model.cl_multi_update(rand, 0.1, winners)
        
        # Make predictions
        pred = model.predict(test_inp)
        # test error
        previous_error = e
        e = model.res_error(pred, test_target)
        epoch += 1
        if (epoch % 5 == 0):
            print("Epoch {}, residual error: {}, dw: {}".format(epoch, e, np.average(model.dw)))
    print("Epoch {}, residual error: {}, dw: {} ".format(epoch, e, np.average(model.dw)))
    return model, e
    


units = 30
width = 0.1
lr = 0.25
epochs = 100
winners_multi = 5
# batch(units, trainset, testset, width)
mod1, noise_nocl = delta(units, width, lr, epochs)
mod2, noise_cl_van = delta(units, width, lr, epochs, winners = 1)
mod3, noise_cl_mul = delta(units, width, lr, epochs, winners = winners_multi)


print("Units: {}. Width: {}. Learning rate: {}. Winners in multi CL: {}".format(units, width, lr, winners_multi))
print("Ballistic data. No CL: {}".format(noise_nocl))
print("Ballistic data. Vanilla CL: {}".format(noise_cl_van))
print("Ballistic data. Multi CL: {}".format(noise_cl_mul))

## Compare units
## Scatter Plot
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(211)
ax.scatter(mod1.units, np.ones(mod1.units.shape)*1.5, marker = "o", c = 'green', label = "No CL")
ax.scatter(mod2.units, np.ones(mod1.units.shape), marker = "o", c = 'red', label = "Vanilla CL")
ax.scatter(mod3.units, np.ones(mod1.units.shape)*0.5, marker = "o", c = 'blue', label = "Multi CL")

ax.set(title='',
        xlabel = "Unit pos",
      ylim = (0, 2))
ax.legend()
plt.show()

train inputs example:
[[ 0.896  0.54 ]
 [ 0.279  0.859]
 [ 0.307  0.403]
 [ 0.974  0.546]
 [ 0.069  0.903]]
[ 0.      0.0448  0.0896  0.1344  0.1792  0.224   0.2688  0.3136  0.3584
  0.4032  0.448   0.4928  0.5376  0.5824  0.6272  0.672   0.7168  0.7616
  0.8064  0.8512  0.896   0.9408  0.9856  1.0304  1.0752  1.12    1.1648
  1.2096  1.2544  1.2992  1.344 ]
Initiated weights: 
[ 0.70369669  0.7904645   0.88295892  0.3671288   0.7116486   0.71358653
  0.4233931   0.07494317  0.76006148  0.19779033  0.86137729  0.7630675
  0.93452646  0.82746502  0.04519396  0.96131806  0.67138817  0.06941045
  0.66830021  0.14351518  0.11236137  0.86337991  0.52413626  0.20603774
  0.2620297   0.56514146  0.18926909  0.55928898  0.42784694  0.82868628]
phi_x:
(2, 30)
self.w
(30,)
Predicted: 
[  2.26751535e+00   5.22370189e-08]
from input:
[ 0.85   0.019]
(3, 30)


ValueError: non-broadcastable output operand with shape (30,) doesn't match the broadcast shape (3,30)